# How to generate publication quality tables


Estimagic can create publication quality tables of parameter estimates in LaTeX or HTML. It works with the results from `estimate_ml` and `estimate_msm` but also supports statsmodels results out of the box. 

You can get almost limitless flexibility if you split the table generation into two steps. The fist generates a DataFrame which you can customize to your liking, the second renders that DataFrame in LaTeX or HTML.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
from IPython.core.display import HTML, Latex

from estimagic import estimation_table, render_html, render_latex
from estimagic.config import EXAMPLE_DIR

## Create tables from statsmodels results

In [2]:
df = pd.read_csv(EXAMPLE_DIR / "diabetes.csv", index_col=0)
mod1 = sm.ols("target ~ Age + Sex", data=df).fit()
mod2 = sm.ols("target ~ Age + Sex + BMI + ABP", data=df).fit()
models = [mod1, mod2]

In [4]:
HTML(estimation_table(models, return_type="html"))

## Adding estimagic results

`estimate_ml` and `estimate_msm` can both generate summaries of estimation results. Those summaries are either DataFrames with the columns `"value"`, `"standard_error"`, `"p_value"` and `"stars"` or pytrees containing such DataFrames. 

For examples, check out our tutorials on [`estimate_ml`](../../getting_started/first_likelihood_estimation_with_estimagic.ipynb) and [`estimate_msm`](../../getting_started/first_msm_estimation_with_estimagic.ipynb).


Assume we got the following DataFrame from an estimation summary:

In [5]:
params = pd.DataFrame(
    {
        "value": [142.123, 51.456, -33.789],
        "standard_error": [3.1415, 2.71828, 1.6180],
        "p_value": [1e-8] * 3,
    },
    index=["Intercept", "Age", "Sex"],
)
params

,value,standard_error,p_value
Intercept,142.123,3.14150,1.000000e-08
Age,51.456,2.71828,1.000000e-08
Sex,-33.789,1.61800,1.000000e-08


You can either use just the params DataFrame or a dictionary containing "params" and additional information in `estimation_table`.

In [6]:
mod3 = {"params": params, "name": "target", "info": {"n_obs": 445}}
models = [mod1, mod2, mod3]

In [7]:
HTML(estimation_table(models, return_type="html"))

## Selecting the right return_type

The following return types are supported:
- `"latex"`: Returns a string that you can save and import into a LaTeX document
- `"html"`: Returns a string that you can save and import into a HTML document.
- `"render_inputs"`: Returns a dictionary with the following entries:
    - `"body"`: A DataFrame containing the main table
    - `"footer"`: A DataFrame containing the statisics
    - other stuff that you should ignore
- `"dataframe"`: Returns a DataFrame you can look at in a notebook

## Use `render_inputs` for maximum flexibility

As an example, let's assume we want to remove a few rows from the footer.

Let's first look at the footer we get from `estimation_table`

In [8]:
render_inputs = estimation_table(models, return_type="render_inputs")
footer = render_inputs["footer"]
footer

target                    
                              (1)            (2)  (3)
Observations                  442            442  445
R$^2$                        0.04           0.40     
Adj. R$^2$                   0.03           0.40     
Residual Std. Error         75.90             60     
F Statistic          8.06$^{***}$  72.90$^{***}$

Now we can remove the rows we don't need and render it to html. 

In [9]:
render_inputs["footer"] = footer.loc[["R$^2$", "Observations"]]
HTML(render_html(**render_inputs))

In [10]:
render_inputs.update({"render_options": {"caption": "Title"}})

## LaTeX peculiarities

By default, tables in `render_latex` are structured in compliance with `siunitx` package. This is done by setting column formats to `S` in the default rendering options defined internally. 
To get nicely formatted tables, you need to add the following to the preamble of your main `.tex` file:
```latex
\sisetup{
        input-symbols            = (),
        table-align-text-post    = false
        group-digits             = false,
    }
```
The first line is necessary if you have parentheses in your table cells (e.g. when displaying inferences), otherwise LaTex will raise an error.

The second argument is necessary so that there is no spacing between the significance stars and the numerical values.

The third line prevents digits in numbers being grouped into groups of threes, which is the default behaviour.
This line is optional, but recommended.

By default, whenever calling `render_latex`, a warning will be raised about this. To silence the warning, set `siunitx_warning=False` in the relvant function calls (when calling `estimation_table` with `return_type=tex` or when calling `render_latex`)

If you don't want to generate `siunitx` style tables, you can pass `render_options={"siunitx:False}` to your function calls. 

You can influence the format of the output table with keyword arguments passed via `render_options`. For the list of supported keyword arguments see [documentation of pandas.io.formats.style.Styler.to_latex](https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html)
- show one example

## Advanced options 

show one example with where many optional arguments are used at once

- options dictionary with all standard entries
- custom param names
- custom col names
- custom col groups
- custom number format
- title

everything that is not in this list will be left to the docstring.

In [13]:
stats_dict = {
    "Observations": "n_obs",
    "R$^2$": "rsquared",
    "Adj. R$^2$": "rsquared_adj",
    "Residual Std. Error": "resid_std_err",
    "F Statistic": "fvalue",
    "show_dof": True,
}